In [1]:
import matplotlib as plt
from matplotlib.pyplot import imshow
import numpy as np
from random import sample
import tensorflow.keras.datasets as datasets
from dnn_tf import dnn
import tensorflow as tf

In [2]:
#Parameters - To be defined by user
nClass = 10
nHidden = [256, 64]
nInput = 784
layers = [nInput] + nHidden + [nClass]
limit = 0.0001
#Hyperparameters - To be tuned by the user
learning_rate = 0.01
nMiniBatch = 128
nIter = 10000
print("Layers: ", layers)
print("Learning Rate: ", learning_rate)
print("Number of MiniBatch: ", nMiniBatch)
print("Iterations: ", nIter)
layers[-1]

Layers:  [784, 256, 64, 10]
Learning Rate:  0.01
Number of MiniBatch:  128
Iterations:  10000


10

In [62]:
#-------data input function-----------------------------#
def getDataset(name, nClass):
    if name=="mnist":
        dataset = datasets.mnist
    (x_train, y_train),(x_test, y_test) = dataset.load_data()     #downloading and loading the dataset
    #x_train, x_test = x_train / 255.0, x_test / 255.0             #normalizing the input data
    x_train_flat = x_train.reshape(x_train.shape[0],-1)         #making dataset suitable for input in Fully Connected layer
    x_test_flat = x_test.reshape(x_test.shape[0],-1)          #making dataset suitable for input in Fully Connected layer
    y_train_onehot = np.eye(nClass)[y_train]                    #converting to one hot vectors
    y_test_onehot = np.eye(nClass)[y_test]                     #converting to one hot vectors
    print(x_train_flat.shape)
    print(y_train_onehot.shape)
    x_train_batch = np.array_split(x_train_flat, int(60000/128))
    print(x_train_batch[2].shape)
    return x_train_flat,x_test_flat,y_train_onehot,y_test_onehot


In [63]:
x_train, x_test, y_train, y_test = getDataset("mnist", nClass)

(60000, 784)
(60000, 10)
(129, 784)


In [5]:
x_train.T.shape
a = np.random.randn(3,3)
print(a)
a.T[1]
z = np.less_equal(a,0.5)
z = (1-z)
z = z * a
print(z)
def fn(z):
    z[1]= z[1]**2
fn(z)
print(z)

[[ 0.29316024  1.35453187  1.03613465]
 [ 1.56323301 -0.2697107   1.21714554]
 [ 0.41315071  0.31256806  2.14785809]]
[[ 0.          1.35453187  1.03613465]
 [ 1.56323301 -0.          1.21714554]
 [ 0.          0.          2.14785809]]
[[0.         1.35453187 1.03613465]
 [2.44369744 0.         1.48144326]
 [0.         0.         2.14785809]]


In [6]:
def pi_membership_function(r,c,radius):
    norm = abs(r-c)
    if radius == 0:
        return np.zeros(norm.shape)
    else:
        for i in range(len(r)):
            if norm[i] <= radius and norm[i] >= radius/2:
                norm[i] = (2*((1-norm[i]/radius)**2))
            elif norm[i] < radius/2 and norm[i] >= 0:
                norm[i] = (1 - 2*((norm[i]/radius)**2))
            else:
                norm[i] = 0
    return norm
    

In [7]:
a = float('nan')
a = a *0
print(a)


nan


In [64]:
def fuzzifyDataset(x_train):
    fdenom = 2
    F_max = np.array([255]*784)
    F_min = np.array([0]*784)
#     F_max = np.ndarray.max(x_train, axis = 0)
#     F_min = np.ndarray.min(x_train, axis = 0)
    lambda_medium = 0.5*(F_max - F_min)
    c_medium = F_min + lambda_medium
#     print(c_medium)
    lambda_low = (1/fdenom)*(c_medium-F_min)
#     print(lambda_low)
    c_low = c_medium - 0.5 * lambda_low
#     print(c_low)
    lambda_high = (1/fdenom) * (F_max - c_medium)
#     print(lambda_high)
    c_high = c_medium + 0.5 * lambda_high
#     print(c_high)
    x_train = x_train.T
#     print(x_train)
    x_train_new = []
    for i in range(len(F_max)):
        x_train_new.append(pi_membership_function(x_train[i],c_low[i],lambda_low[i]))
        x_train_new.append(pi_membership_function(x_train[i],c_medium[i],lambda_medium[i]))
        x_train_new.append(pi_membership_function(x_train[i],c_high[i],lambda_high[i]))
    return x_train_new

In [65]:
import time
start = time.time()
x_train_fuzzy = fuzzifyDataset(x_train[3:5])
end = time.time()

In [66]:
x_train_fuzzy = np.array(x_train_fuzzy)


In [67]:
for i in x_train_fuzzy:
    print(i)


[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]


[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0.05425606 0.        ]
[0. 0.]
[0. 0.]
[0.00196847 0.        ]
[0. 0.]
[0. 0.]
[0.00196847 0.        ]
[0. 0.]
[0.         0.70158401]
[0.42826605 0.60725875]
[0.36208381 0.        ]
[0. 0.]
[0.01771626 0.00049212]
[0. 0.]
[0. 0.]
[0.         0.00110727]
[0. 0.]
[0. 0.]
[0.         0.05425606]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0.00049212 0.        ]
[0. 0.]
[0. 0.]
[0.00196847 0.        ]


In [15]:
print(end-start)
def main():
    x_train, x_test, y_train, y_test = getDataset("mnist", nClass)
    test = dnn(layers, x_train, x_test, y_train, y_test, nMiniBatch, learning_rate=0.01, training_epochs=10)
    test.initialize()
    test.forwardProp()
    test.train()
    #test.test()
    
    

99.407790184021


In [10]:
if __name__ == "__main__":
    main()

(60000, 784)
(60000, 10)
(129, 784)
(?, 784)
(?, 10)
Epoch: 0001 cost= 18.088355462
Optimization Finished!
Test Accuracy: 0.9054


In [17]:
print(tf.__version__)

1.12.0
